### Test ML Embeddings

In [1]:
from parse import get_data
import pandas as pd
import numpy as np

########################### EXTRACT DATA

def convert_labels(l) :
    if l == 'irr' :
        return [1, 0, 0, 0]
    elif l == 'neg' :
        return [0, 1, 0, 0]
    elif l == 'neu' :
        return [0, 0, 1, 0]
    else :
        return [0, 0, 0, 1]
    
def decode_output(out_array) :
    if out_array[0] == np.max(out_array) :
        return "irr", np.max(out_array)
    elif out_array[1] == np.max(out_array) :
        return "neg", np.max(out_array)
    elif out_array[2] == np.max(out_array) :
        return "neu",np.max(out_array)
    elif out_array[3] == np.max(out_array) :
        return "pos",np.max(out_array)
    else :
        return "err",0

text, label = get_data("corpus_ready.data")
labels = ["irr", "neg", "neu", "pos"]
label_data = np.array([convert_labels(l) for l in label])
print("size of train data : ", len(text))

######################### VECTORIZE DATA

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from functools import reduce

estimated_vocab_size = 2000
encoded = [one_hot(t, estimated_vocab_size) for t in text]
maxlength = np.max([len(x) for x in encoded])
encoded = pad_sequences(encoded, maxlen=maxlength, padding='post')

size of train data :  3309


Using TensorFlow backend.


In [5]:
percentage_train = 0.8
borne = int(percentage_train*len(text))
X_train = encoded[:borne]
Y_train = label_data[:borne]
X_test = encoded[borne:]
Y_test = label_data[borne:]

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(input_dim=estimated_vocab_size, output_dim=8, input_length=maxlength))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# print(model.summary())
batch = 100
model.fit(X_train, Y_train, epochs=50, verbose=1)

Epoch 1/50
2647/2647 [==============================] - 0s 73us/step - loss: 0.5485 - acc: 0.7710
Epoch 2/50
2647/2647 [==============================] - 0s 35us/step - loss: 0.4499 - acc: 0.7900
Epoch 3/50
2647/2647 [==============================] - 0s 32us/step - loss: 0.4275 - acc: 0.8092
Epoch 4/50
2647/2647 [==============================] - 0s 40us/step - loss: 0.4013 - acc: 0.8289
Epoch 5/50
2647/2647 [==============================] - 0s 35us/step - loss: 0.3759 - acc: 0.8436
Epoch 6/50
2647/2647 [==============================] - 0s 38us/step - loss: 0.3532 - acc: 0.8566
Epoch 7/50
2647/2647 [==============================] - 0s 64us/step - loss: 0.3338 - acc: 0.8687
Epoch 8/50
2647/2647 [==============================] - 0s 80us/step - loss: 0.3155 - acc: 0.8786
Epoch 9/50
2647/2647 [==============================] - 0s 34us/step - loss: 0.2979 - acc: 0.8872
Epoch 10/50
2647/2647 [==============================] - 0s 36us/step - loss: 0.2813 - acc: 0.8916
Epoch 11/50
2647/26

In [7]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
# loss, accuracy = model.evaluate(X_train, Y_train)
print(accuracy)

0.7571752269462516


In [10]:
test = "J'aime la glace à la banane"
test_oh = [one_hot(test, estimated_vocab_size)]
test_oh = pad_sequences(test_oh, maxlen=maxlength, padding='post')
print(type(test_oh))
print(test_oh)
out = list(model.predict(test_oh)[0])
decode_output(out)
# print("tweet is evaluated " + out[0] + " at " + str(out[1]) + "%")

<class 'numpy.ndarray'>
[[1547  558  681 1873  558 1827    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]


('neu', 0.48220602)